In [31]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [20]:
filtered_vocab_df = pd.read_json("../data/filtered_vocab_df.json")
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
2313,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921,True,"say, call, tell"
4254,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680,True,"book, volume, inner bark of a tree, book, book"
10443,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563,True,"be able, can"
6100,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367,True,"do, make, handle"
6099,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310,True,"have, hold, possess, consider, think"
1764,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397,0.002279,True,part
8953,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,...,0.001686,0.001846,0.002739,0.002701,0.002085,0.002239,0.002556,0.002129,True,suddenly
1449,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,...,0.001883,0.001437,0.001897,0.002244,0.001466,0.001554,0.001730,0.001717,True,
2323,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,...,0.001626,0.001578,0.001717,0.001703,0.002230,0.001240,0.001090,0.001632,True,"place, location"
56,aqua,11747,41231,20765,34038,42778,18714,30884,14948,28253,...,0.001232,0.000754,0.001905,0.001437,0.001160,0.001966,0.005758,0.001575,True,"get/fetch/bring water, be watered"


In [21]:
filtered_vocab_df["shared"].sum()

3055

In [22]:
with open("../data/vectors_dict_comp.pkl", "rb") as file:
    vectors_dict = pickle.load(file)

In [23]:
vectors_dict.keys()

dict_keys(['NOSCEMUS - 1501-1550', 'NOSCEMUS - 1551-1600', 'NOSCEMUS - 1601-1650', 'NOSCEMUS - 1651-1700', 'NOSCEMUS - Alchemy/Chemistry', 'NOSCEMUS - Astronomy/Astrology/Cosmography', 'NOSCEMUS - Biology', 'NOSCEMUS - Geography/Cartography', 'NOSCEMUS - Mathematics', 'NOSCEMUS - Medicine', 'NOSCEMUS - Meteorology/Earth sciences', 'NOSCEMUS - Physics', 'EMLAP', 'LASLA', 'Opera Maiora'])

In [24]:
shared_vocabulary = filtered_vocab_df[filtered_vocab_df["shared"]]["word"].tolist()

In [25]:
def cosine_matrix_for_kv(kv, words):
    """Return (n x n) cosine similarity matrix for the given KeyedVectors and word list."""
    # Stack vectors in the *same fixed order* across subcorpora
    X = np.vstack([kv[w] for w in words]).astype(np.float32)  # (n, d)
    # Normalize rows
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    # Guard against any rare zero vectors
    norms[norms == 0] = 1.0
    Xn = X / norms
    # Cosine sim = dot of normalized rows
    return Xn @ Xn.T  # (n, n), symmetric, diag=1

# Build the dict of matrices
cos_sim_matrices_dict = {}
for name, kv in vectors_dict.items():
    # Optional: assert all words present (paranoia, even if you said they are shared)
    missing = [w for w in shared_vocabulary if w not in kv]
    if missing:
        raise ValueError(f"{name}: {len(missing)} words missing from shared_vocabulary (e.g., {missing[:5]})")
    cos_sim_matrices_dict[name] = cosine_matrix_for_kv(kv, shared_vocabulary)

# (Optional) store as float16 to save RAM/disk (lossy but often fine for cosine sims)
cos_sim_matrices_dict_f16 = {k: v.astype(np.float16) for k, v in cos_sim_matrices_dict.items()}

In [47]:
with open("../data/cos_sim_matrices_dict_f16.pkl", "wb") as f:
    pickle.dump(cos_sim_matrices_dict_f16, f)

In [48]:
with open("/home/jupyter-vojta/notebooks/iweems_huggingface/data/cos_sim_matrices_dict_f16.pkl", "wb") as f:
    pickle.dump(cos_sim_matrices_dict_f16, f)

In [49]:
with open("/srv/webserver/apps/iweems_app/data/cos_sim_matrices_dict_f16.pkl", "wb") as f:
    pickle.dump(cos_sim_matrices_dict_f16, f)

In [26]:
cos_sim_matrices_dict_f16["EMLAP"]

array([[1.    , 0.5127, 0.5684, ..., 0.3274, 0.2507, 0.4592],
       [0.5127, 1.    , 0.3152, ..., 0.1582, 0.2644, 0.3013],
       [0.5684, 0.3152, 1.    , ..., 0.2864, 0.244 , 0.3896],
       ...,
       [0.3274, 0.1582, 0.2864, ..., 1.    , 0.072 , 0.2198],
       [0.2507, 0.2644, 0.244 , ..., 0.072 , 1.    , 0.2634],
       [0.4592, 0.3013, 0.3896, ..., 0.2198, 0.2634, 1.    ]],
      dtype=float16)

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def term_correlation_matrix(cos_dict_f16, vocabulary, term, order=None):
    if term not in vocabulary:
        raise ValueError(f"Term '{term}' not in shared vocabulary.")
    term_idx = vocabulary.index(term)

    names = order if order is not None else list(cos_dict_f16.keys())
    rows = []
    for name in names:
        M = cos_dict_f16[name].astype(np.float32, copy=False)
        r = M[term_idx, :]
        r = np.concatenate([r[:term_idx], r[term_idx+1:]])  # drop self
        rows.append(r)

    X = np.vstack(rows)  # (N, V-1)

    # Pearson; handle zero-variance rows -> NaNs
    C = np.corrcoef(X)
    # Keep diagonal = 1 and replace remaining NaNs with 0 (or your preference)
    np.fill_diagonal(C, 1.0)
    C = np.nan_to_num(C, nan=0.0)

    return pd.DataFrame(C, index=names, columns=names)

def from_term_to_corr_plot(cos_sim_matrices_dict_f16, shared_vocabulary, target_term, order=None, return_df=False):
    try:
        corr_df = term_correlation_matrix(cos_sim_matrices_dict_f16, shared_vocabulary, target_term, order=order)

        n = len(corr_df)
        # Scale figure a bit with n (capped)
        fw = min(1.0 + 0.5 * n, 14)
        fh = min(1.0 + 0.4 * n, 10)

        fig, ax = plt.subplots(figsize=(fw, fh))
        im = ax.imshow(corr_df.values, vmin=-1, vmax=1, interpolation="nearest")
        ax.set_title(f"Pearson correlation of '{target_term}' similarity profile across subcorpora")
        ax.set_xticks(range(n)); ax.set_yticks(range(n))
        ax.set_xticklabels(corr_df.columns, rotation=45, ha="right")
        ax.set_yticklabels(corr_df.index)
        ax.grid(False)

        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label("Pearson r")

        plt.tight_layout()
        return (fig, corr_df) if return_df else fig
    except:
        print(f"The term '{target_term}' is not in the shared vocabulary present in all subcorpora..")

In [46]:
target_term = "sciena"
fig = from_term_to_corr_plot(cos_sim_matrices_dict_f16, shared_vocabulary, target_term)

The term 'sciena' is not in the shared vocabulary present in all subcorpora..


In [50]:
cos_sim_matrices_dict_f16.keys()

dict_keys(['NOSCEMUS - 1501-1550', 'NOSCEMUS - 1551-1600', 'NOSCEMUS - 1601-1650', 'NOSCEMUS - 1651-1700', 'NOSCEMUS - Alchemy/Chemistry', 'NOSCEMUS - Astronomy/Astrology/Cosmography', 'NOSCEMUS - Biology', 'NOSCEMUS - Geography/Cartography', 'NOSCEMUS - Mathematics', 'NOSCEMUS - Medicine', 'NOSCEMUS - Meteorology/Earth sciences', 'NOSCEMUS - Physics', 'EMLAP', 'LASLA', 'Opera Maiora'])

In [ ]:
['LASLA', 'Opera Maiora', 'NOSCEMUS - 1501-1550', 'NOSCEMUS - 1551-1600', 'NOSCEMUS - 1601-1650', 'NOSCEMUS - 1651-1700', 'NOSCEMUS - Alchemy/Chemistry', 'NOSCEMUS - Astronomy/Astrology/Cosmography', 'NOSCEMUS - Biology', 'NOSCEMUS - Geography/Cartography', 'NOSCEMUS - Mathematics', 'NOSCEMUS - Medicine', 'NOSCEMUS - Meteorology/Earth sciences', 'NOSCEMUS - Physics', 'EMLAP']

In [ ]:

# Save
with open("../data/cos_sim_matrices_dict.pkl", "wb") as f:
    pickle.dump({
        "vocabulary": shared_vocabulary,          # keep the order alongside
        "matrices": cos_sim_matrices_dict_f16     # or use cos_sim_matrices_dict for float32
    }, f)

# Usage later:
# with open("../data/cos_sim_matrices_dict.pkl", "rb") as f:
#     saved = pickle.load(f)
# shared_vocabulary = saved["vocabulary"]
# cos_sim_matrices_dict = saved["matrices"]
# emlap_matrix = cos_sim_matrices_dict["EMLAP"]  # shape: (len(shared_vocabulary), len(shared_vocabulary))